In [11]:
# %conda install cudatoolkit
!nvidia-smi


Thu May  2 15:57:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   40C    P0              61W / 300W |   4640MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Notebook by Jose described in 
https://www.notion.so/meetcleo/Working-with-open-source-LLMs-78dcf0cb0fda48ce9393257dfd77c485#112aa21039a84340866ed4f7798da10c



In [ ]:
%pip install pigar

In [1]:
%pip install --upgrade torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qoiy_lnm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-qoiy_lnm
  Resolved https://github.com/huggingface/transformers to commit 39359e5b5f6246c667cc25f743437cb05d492ea7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install --upgrade mlflow

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())

In [3]:
import pandas as pd
import os

import numpy as np
import pandas as pd
import torch
from peft import PeftModel
import mlflow

In [5]:
!pigar generate

Requirements file has been overwritten, no difference.
Requirements has been written to /home/sagemaker-user/exploration/MISTRAL_JOSE/requirements.txt.


In [3]:
!pipreqs /home/sagemaker-user/exploration/MISTRAL_JOSE/

INFO: Not scanning for jupyter notebooks.


### Load Data

In [4]:
test_df = pd.read_parquet("s3://cleo-data-science/mlflow/40/4208e04c9f0e442bbff73bd6996bcf8b/artifacts/input_data/test.parquet")

In [5]:
test_df.head()

,Unnamed: 0.1,Unnamed: 0,index,bot_request_id,bot_request_created_date,first_annotation_date,classifier_label,num_annotations,company_annotated,max_annotations_per_intent,max_agreement,agreed_label,normalised_bot_request_body,prompt,completion
0,18134,18134,18134,223796853,2023-05-13,2023-05-20,send_overdraft_protection,4,cleo,3,0.750000,send_overdraft_protection,Can I get a advance,USER: Can I get a advance\n\n###\n\n,86
1,9123,9123,9123,223118006,2023-05-01,2023-05-11,send_overdraft_protection,4,cleo,4,1.000000,send_overdraft_protection,Loan,USER: home\nCLEO: unknown\nUSER: home\nCLEO: u...,86
2,18151,18151,18151,223797771,2023-05-13,2023-05-15,send_add_account,4,cleo,4,1.000000,atm,Cleo affiliated atm,USER: Can I use Cleo card to withdraw funds at...,1
3,17639,17639,17639,223767952,2023-05-12,2023-05-15,send_faq,3,cleo,2,0.666667,send_help,Help,"USER: Debit card\nCLEO: No problem, Rochelle. ...",66
4,9919,9919,9919,223183772,2023-05-02,2023-05-11,send_unsubscribe_from_premium,4,cleo,4,1.000000,send_unsubscribe_from_premium,Unsubscribe now please,USER: I would like to speak to an actual perso...,114


### Load model

In [6]:

from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    MistralForSequenceClassification,
)

2024-05-02 15:50:16.012254: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://mlflow.internal.platform.cleoites.tech"
# Copy and paste the token from http://espresso-gateway.mikeverse.cleoites.tech/
os.environ["MLFLOW_TRACKING_TOKEN"] = "eyJraWQiOiJaYk93dnR4ZXU0cDFqdDd6R1NhQVhxSERWN2tcL1N1OVlVa0lQem1kcWxiaz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJkYmM3ZDJhMC1jNmYyLTQ3ZDYtOGVhOS1iOGIwZWIxMjVhNTciLCJjb2duaXRvOmdyb3VwcyI6WyJmZWFzdC11c2VycyIsIm1sZmxvdy11c2VycyIsImVzcHJlc3NvLXJlcXVlc3RlcnMiLCJkYXRhLXNjaWVudGlzdHMiLCJ1cy1lYXN0LTFfd1RZVFVQbkdOX2NsZW8tZW1wbG95ZWVzIl0sImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy1lYXN0LTEuYW1hem9uYXdzLmNvbVwvdXMtZWFzdC0xX3dUWVRVUG5HTiIsInZlcnNpb24iOjIsImNsaWVudF9pZCI6IjdlbHUxNzBxdDJvYmNsMWhhYTRuaHBqbHRkIiwib3JpZ2luX2p0aSI6ImZjMjBhMGEyLThjNGMtNGE5MC1hN2I0LTNlMjkxZWJlN2VhNiIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoib3BlbmlkIiwiYXV0aF90aW1lIjoxNzE0NTc5MDI3LCJleHAiOjE3MTQ2NzUxNzksImlhdCI6MTcxNDY0NjM3OSwianRpIjoiZjJiYzc4NDEtNDE2ZC00YWU4LTg4NmYtYzkyYTExZDY1NGJlIiwidXNlcm5hbWUiOiJjbGVvLWVtcGxveWVlc19jbGFyYS5sQG1lZXRjbGVvLmNvbSJ9.la5KCgACD73tnPV_ERynavoOvcR5ULvfnEOqhiyW-DJRjlvauhs1-_4mRx4vj4y0UuyYwdiNJsITDldWr42NpcbLQea1Yc0l5lEbt_KeRyigSfCk-mG0MRVJ0e7hbNk2Xgh_NqW7j8Lcs4Ov10uefd4jqxw-vC40BOuPLNRd39-ec4fJZ74YFtOAV3Qk5KiI6u3aWUhkoqLyt76NkwGLz7qNG1G3Gt_rQcxQUzcQVXRaRsHZM3aZ40o0kllRDhgO33Xgo0aKC9JWyhdoQ9_7jsar8f1mvbmYDLeG3GX8fcANNlm1cHcppk7jHcRjnF_3t_MJFdD3fSCuI7a6J3exKQ"

In [9]:
mlflow.artifacts.download_artifacts(
        run_id="16f4aab31bc8455482c632846a19c0cb",
        artifact_path="checkpoint-2950/artifacts/checkpoint-2950",
        dst_path="model",
    )

KeyboardInterrupt: 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
        base_model_id, model_max_length=512, padding_side="left", add_eos_token=True
    )

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
ft_model = PeftModel.from_pretrained(
    base_model,
    "model/checkpoint-2950/artifacts/checkpoint-2950",
)

In [ ]:
model_input = tokenizer(
            test_df["prompt"][0],
            truncation=True,
            max_length=512,
            padding="max_length",
            return_tensors="pt",
        ).to("cuda")

In [ ]:
with torch.no_grad():
    outputs = ft_model(**model_input).logits
    # Get the list of class probabilities
    pred = torch.softmax(outputs, dim=1).tolist()[0]
    # Append the highest probability
    pred_class = np.argmax(pred, axis=-1)

In [ ]:
pred_class

In [ ]:
test_df["completion"][0]

In [7]:
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [8]:
base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        low_cpu_mem_usage=True
)   
base_model = MistralForSequenceClassification.from_pretrained(
        base_model_id,
        num_labels=128,
        quantization_config=bnb_config,
    )

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
